In [1]:

import numpy as np
from time import sleep
import googlemaps
import folium
import pandas as pd
import getpass
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [2]:
load_dotenv
maps_token = getpass.getpass("maps_token: ")
map_client = googlemaps.Client(maps_token)

In [3]:
def extracting_data(business,keyword, radius):
    sleep(1)
    location = (51.5053503,-0.0236837)  
    radius = radius # radius in meters
    business = business
    keyword = keyword
    results = []
    response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword)
    results.extend(response['results'])
    while True:
        try:
            sleep(2)
            response = map_client.places_nearby(location=location, type=business, radius=radius, keyword=keyword, page_token=next_page)
            results.extend(response['results'])
            next_page = response['next_page_token']
        except:
            break
    return results    

In [4]:
def dataframe(data, business, keyword, color, icon_color, icon):
    list_local = []
    df = pd.DataFrame(columns = ['Category','Type', 'Name', 'Address', 'Latitude', 'Longitude', 'Icon', 'Icon Color', 'Color', 'Prefix'])
    for result in data:
        local = {}
        local['Category'] = business
        local['Type'] = keyword
        local['Color'] = color
        local['Icon Color'] = icon_color
        local['Icon'] = icon
        local['Prefix'] = 'fa'
        local['Name'] = result['name']
        local['Address'] = result['vicinity']
        local['Latitude'] = result['geometry']['location']['lat']
        local['Longitude'] = result['geometry']['location']['lng']
        list_local.append(local)
    df = pd.DataFrame(list_local)
    return df    

In [5]:
bars = extracting_data('Bar', 'Bar', 1500)

In [6]:
restaurant = extracting_data('Restaurant', 'Restaurant', 1000)

In [7]:
school = extracting_data('School', 'public School', 2000)

In [8]:
veg_restaurant = extracting_data('Restaurant', 'vegetarian', 500)

In [9]:
hairdresser = extracting_data('hairdresser', 'hairdresser', 300)

In [10]:
tech_startups = extracting_data('tech startups', 'startups', 2000)

In [11]:
coffee = extracting_data('Coffe', 'Coffee Shop', 500)

In [12]:
metro = extracting_data('Metro', 'Metro Stations', 700)

In [13]:
bus = extracting_data('Bus', 'Bus Station', 700)

In [14]:
travel_agency = extracting_data('Travel Agency', 'Travel Agency', 500)

In [15]:
df_bars_london = dataframe(bars,'Bar', 'Bar', 'red', 'white', 'beer-mug-empty' )

In [16]:
df_restaurants_london = dataframe(restaurant,'Restaurant', 'Restaurant', 'orange', 'white', 'utensils')

In [17]:
df_school_london = dataframe(school, 'School', 'Public School', 'blue', 'black', 'graduation-cap')

In [18]:
df_veg_london = dataframe(veg_restaurant,'Restaurant', 'Vegetarian', 'green', 'white', 'wheat-awn')

In [19]:
df_hairdresser_london = dataframe(hairdresser, 'Hairdresser', 'Health & Beauty', 'white', 'black', 'scissors')

In [20]:
df_tech_startups_london = dataframe(tech_startups, 'Tech startups', 'Startups', 'black', 'white', 'desktop')

In [21]:
df_coffee_london = dataframe(coffee, 'Coffee', 'Coffee  Shop', 'beige', 'back', 'mug-hot')

In [22]:
df_metro_london = dataframe(metro, 'Metro', 'Metro Station', 'darkred', 'white', 'train-subway')

In [23]:
df_bus_london = dataframe(bus, 'Bus', 'Bus Station', 'darkblue', 'white', 'bus')

In [24]:
df_travel_agency_london = dataframe(travel_agency, 'Travel Agency', 'Travel Agency', 'purple', 'black', 'plane')

# Exporting the DataFrame to Mongodb

In [25]:
def export_to_mongodb(data):
    client = MongoClient("localhost", 27017)
    db = client["Project-Spot"]
    for i, v in enumerate(data):
        file_csv = f"Project-Spot_{i}.csv"
        v.to_csv(file_csv, index=False)
        collection_name = v['Type'].iloc[0]
        collection = db[collection_name]
        result = v.to_dict(orient='records')
        collection.insert_many(result)
    

# Create the Map layer and add it to the FeatureGroup

In [26]:
city_map = [
    df_coffee_london, df_tech_startups_london,
    df_hairdresser_london, df_veg_london, 
    df_school_london, df_bars_london,
    df_restaurants_london, df_metro_london,
    df_bus_london, df_travel_agency_london
]

layers_name = [
    'Coffee Shop', 'Startups',
    'Health & Beauty', 'Vegetarian',
    'Public School', 'Bar',
    'Restaurant', 'Metro Station',
    'Bus Station', 'Travel Agency'
]

In [27]:
export_to_mongodb(city_map)

In [34]:
def add_spot(data, layer):
    main_map = folium.Map(location = ['51.5053503','-0.0236837'], tiles="Cartodb dark_matter", zoom_start=15)
    layer_list = []
    red_cicle = folium.FeatureGroup(name = 'Red Cicle')    
    folium.Circle(location=['51.5053503','-0.0236837'], radius=2500, color='red', fill=True).add_to(red_cicle)
    layer_list.append(red_cicle)
    yellow_cicle = folium.FeatureGroup(name = 'Yellow Cicle')    
    folium.Circle(location=['51.5053503','-0.0236837'], radius=1500, color='yellow', fill=True).add_to(yellow_cicle)
    layer_list.append(yellow_cicle)
    green_cicle = folium.FeatureGroup(name = 'Green Cicle')    
    folium.Circle(location=['51.5053503','-0.0236837'], radius=700, color='green', fill=True).add_to(green_cicle)
    layer_list.append(green_cicle)
    for df, lay in zip(data, layer):
        layer = folium.FeatureGroup(name = f'{lay}')
        for k, v in df.iterrows():
            icon = folium.Icon(
                color= v['Color'], 
                icon_color= v['Icon Color'], 
                icon= v['Icon'], prefix= v['Prefix']
                )
            folium.Marker(location=[v['Latitude'], v['Longitude']], popup= v['Name'], icon = icon).add_to(layer)
        layer_list.append(layer)
    for v in layer_list:
        v.add_to(main_map)
    folium.LayerControl().add_to(main_map)
    main_map.save('london_map.html')
    return main_map

In [35]:
add_spot(city_map, layers_name)